In [1]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [2]:
import os
from langsmith_tracker import langsmith

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
langsmith(project_name=os.getenv("LANGCHAIN_PROJECT"))

Langsmith 추적이 활성화되었습니다. [프로젝트명: langchain_note_2024]


### PromptTemplate

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

### 1. form_template() 를 사용한 prompt 생성 방법입니다.

- 치환될 변수를 `{ 변수 }` 로 묶어서 템플릿을 사용합니다.

In [7]:
from langchain_core.prompts import PromptTemplate

template = "{player} 는 어느 팀 소속의 선수인가요?"

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['player'], input_types={}, partial_variables={}, template='{player} 는 어느 팀 소속의 선수인가요?')

`Player` 변수에 값을 넣어 프롬프트를 완성할 수 있습니다.

In [8]:
prompt = prompt.format(player="리오날 메시")
prompt

'리오날 메시 는 어느 팀 소속의 선수인가요?'

In [43]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

template = """
{player} 는 어느 팀 소속의 선수인가요?
[FORMAT] 을 참고하여 답변해주세요.

# FORMAT
선수이름: 
종목: 
소속팀:
연봉정보:
"""

prompt = PromptTemplate.from_template(template)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm
answer = chain.invoke({"player": "리오넬 메시"})
print(answer.content)

선수이름: 리오넬 메시  
종목: 축구  
소속팀: 인터 마이애미 CF  
연봉정보: 약 4,000만 달러 (2023년 기준)  


### 2. PromptTemplate 객체 생성과 동시에 prompt를 생성하는 방법입니다.

- input_variables 을 지정합니다.

In [14]:
template = """
{player} 는 어느 팀 소속의 선수인가요?
[FORMAT] 을 참고하여 답변해주세요.

# FORMAT
선수이름: 
종목: 
소속팀:
연봉정보:
"""

prompt = PromptTemplate(template=template, input_variables=["player"])

chain = prompt | llm
answer = chain.invoke({"player": "리오넬 메시"})
print(answer.content)

선수이름: 리오넬 메시  
종목: 축구  
소속팀: 인터 마이애미 CF  
연봉정보: 약 4천만 달러 (2023년 기준)  


`partial_variables` : 처음에는 변동이 되는 변수이거나 변동이 예상되는 변수였지만, 확정되었을때 사용할 수 있는 문법입니다.

In [17]:
template = "{player} 의 {question1} 과(와) {question2} 정보를 알려주세요."

prompt = PromptTemplate(
    template=template,
    input_variables=["player", "question1"],
    partial_variables={"question2": "수상경력"},
)

prompt

PromptTemplate(input_variables=['player', 'question1'], input_types={}, partial_variables={'question2': '수상경력'}, template='{player} 의 {question1} 과(와) {question2} 정보를 알려주세요.')

In [18]:
prompt.format(player="리오넬 메시", question1="소속팀")

'리오넬 메시 의 소속팀 과(와) 수상경력 정보를 알려주세요.'

`from_template` 로 `patial` 을 사용하는 방법입니다.

In [28]:
prompt = PromptTemplate.from_template(template)
prompt_partial = prompt.partial(question2="수상경력")
prompt_partial

PromptTemplate(input_variables=['player', 'question1'], input_types={}, partial_variables={'question2': '수상경력'}, template='{player} 의 {question1} 과(와) {question2} 정보를 알려주세요.')

In [29]:
prompt_partial.format(player="리오넬 메시", question1="소속팀")

'리오넬 메시 의 소속팀 과(와) 수상경력 정보를 알려주세요.'

In [30]:
chain = prompt_partial | llm
answer = chain.invoke({"player": "리오넬 메시", "question1": "소속팀"})
print(answer.content)

리오넬 메시(Lionel Messi)는 2021년부터 파리 생제르맹(Paris Saint-Germain, PSG) 소속으로 활동하고 있습니다. 이전에는 FC 바르셀로나에서 오랜 기간 동안 활약했습니다.

메시의 주요 수상 경력은 다음과 같습니다:

### 개인 수상
- **발롱도르**: 7회 수상 (2009, 2010, 2011, 2012, 2015, 2019, 2021)
- **FIFA 올해의 선수**: 1회 수상 (2019)
- **UEFA 올해의 선수**: 1회 수상 (2011)
- **Golden Shoe**: 6회 수상 (2010, 2012, 2013, 2017, 2018, 2019)

### 클럽 수상 (FC 바르셀로나)
- **라리가**: 10회 우승
- **코파 델 레이**: 7회 우승
- **UEFA 챔피언스리그**: 4회 우승
- **슈퍼코파 데 에스파냐**: 8회 우승
- **FIFA 클럽 월드컵**: 3회 우승
- **UEFA 슈퍼컵**: 3회 우승

### 국가대표 수상 (아르헨티나)
- **코파 아메리카**: 1회 우승 (2021)
- **FIFA 월드컵**: 1회 우승 (2022)
- **FIFA U-20 월드컵**: 1회 우승 (2005)
- **올림픽 금메달**: 1회 수상 (2008)

메시는 축구 역사상 가장 위대한 선수 중 한 명으로 평가받고 있으며, 그의 경력은 계속해서 기록을 갱신하고 있습니다.


### `partial_variables` : 부분변수 채움

- partial 을 자주 사용하는 용도는 함수 호출할 때입니다. 항상 동일한 방식으로 가져오고 싶은 변수가 있는 경우에 자주 사용합니다.
- 대표적으로 날짜와 시간입니다. 또는 포맷이 있는 경우 사용합니다.

In [32]:
from datetime import datetime


def get_today():
    return datetime.now().strftime("%B %d")


get_today()

'November 11'

In [58]:
prompt = PromptTemplate(
    template="""
    모르거나 없는 정보라면 '정보가 없습니다.' 라고 답변해주세요. 유명인 정보를 나열하지 말아주세요.
    {today} 와 같은 날짜에 생일인 한국의 유명인 {n} 명을 나열해주세요.
    
    # FORMAT
    이름(생일)
    """,
    input_variables=["n"],
    partial_variables={"today": get_today},
)

chain = prompt | llm
answer = chain.invoke({"n": 3})
# answer = chain.invoke({"today":"Jan 02", "n": 3}) # today를 작성해주면 partial이 설정되어 있어도 내용을 덮어 쓰게 됩니다.
print(answer.content)

정보가 없습니다.


### 파일로부터 template 읽어오는 방법입니다.

In [89]:
from langchain_core.prompts import load_prompt

prompt = load_prompt("prompts/pocketmon_pokedex_template.yml")
prompt

PromptTemplate(input_variables=['pocketmon_name'], input_types={}, partial_variables={}, template="{pocketmon_name} 에 대한 정보를 알려주세요.\n포켓몬의 특징을 다음의 [FORMAT]에 맞게 정리하여 답변해주세요.\n한글로 작성해주세요.\n존재하지 않는 포켓몬일 경우 '{pocketmon_name}에 대한 정보를 찾을 수 없습니다.'라고 답변해주세요.\n---\n[FORMAT]\n이름:\n속성:\n유형:\n진화 단계:\n주요 스킬:\n설명:\n체력:\n공격력:\n방어력:\n")

In [90]:
print(prompt.format(pocketmon_name="피카츄"))

피카츄 에 대한 정보를 알려주세요.
포켓몬의 특징을 다음의 [FORMAT]에 맞게 정리하여 답변해주세요.
한글로 작성해주세요.
존재하지 않는 포켓몬일 경우 '피카츄에 대한 정보를 찾을 수 없습니다.'라고 답변해주세요.
---
[FORMAT]
이름:
속성:
유형:
진화 단계:
주요 스킬:
설명:
체력:
공격력:
방어력:



In [91]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm
answer = chain.invoke({"pocketmon_name": "피카츄"})
print(answer.content)

이름: 피카츄  
속성: 전기  
유형: 1세대 포켓몬  
진화 단계: 피카츄 → 라이츄 (진화)  
주요 스킬: 전기쇼크, 백만볼트, 스파크, 번개  
설명: 피카츄는 포켓몬 시리즈의 대표적인 캐릭터로, 귀여운 외모와 전기 속성을 가진 포켓몬입니다. 주로 전기를 저장하고 방출하는 능력을 가지고 있으며, 애니메이션과 게임에서 주인공의 파트너로 등장합니다.  
체력: 35  
공격력: 55  
방어력: 40  


### ChatPromptTemplate

- `ChatPromptTemplate` 는 대화방식의 챗봇을 사용할때는 `PromptTemplate` 보다 조금 더 좋은 성능으로 사용할 수 있습니다.
- AI 와 Human 이 대화를 주고 받는 형식일때 사용합니다.
- `PromptTemplate` 는 지시사항일때 사용합니다.
  
- 메시지는 튜플(tuple)형식으로 구성하며, (role, message) 형태로 작성합니다. 리스트로 생성할 수 있습니다.

##### role

- `system`: 시스템 설정 메세지이며, 전역 설정과 관련한 프롬프트입니다.
- `human`: 사용자 입력 메세지 프롬프트입니다.
- `ai`: AI 의 답변에 관련한 프롬프트입니다.

In [1]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template(
    "{player} 은(는) 어느 팀 소속의 선수인가요?"
)
chat_prompt.format(player="손흥민")

'Human: 손흥민 은(는) 어느 팀 소속의 선수인가요?'

In [2]:
chat_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 친절한 AI 어시드턴트입니다. 당신의 이름은 모비딕이고 직업은 {job} 입니다.",
        ),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    job="pro gammer", user_input="당신의 직업은 무엇입니까?"
)
messages

[SystemMessage(content='당신은 친절한 AI 어시드턴트입니다. 당신의 이름은 모비딕이고 직업은 pro gammer 입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='당신의 직업은 무엇입니까?', additional_kwargs={}, response_metadata={})]

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

llm.invoke(messages).content

'제 직업은 프로 게이머입니다. 게임을 하면서 수입을 올리는 일을 하고 있어요. 어떤 게임을 좋아하시나요? 함께 이야기 나누면 즐거울 것 같아요.'

In [4]:
chain = chat_template | llm
chain.invoke({"job": "pro gammer", "user_input": "당신의 직업은 무엇입니까?"}).content

'제 직업은 프로 게이머입니다. 게임을 하면서 수입을 올리는 일을 하고 있어요. 어떤 게임을 좋아하시나요? 함께 이야기 나누면 즐거울 것 같아요.'

### MessagesPlaceHolder

- 아직 확정된 메세지가 아니지만 나중에 언젠가는 채워질 메시지를 PlaceHolder 로 잡아둡니다.
- 챗봇에서 대화기록을 할 때 사용합니다. 대화기록은 대화중에 확정이 되지 않은 정보이기에 PlaceHolder 를 활용하여 기록합니다.

In [5]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}"),
    ]
)

prompt

ChatPromptTemplate(input_variables=['history', 'question'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchai

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage


# LLM 모델 초기화 (OpenAI의 ChatGPT를 사용하는 예시)
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 대화 기록 생성
history = [
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="I'm doing well, thank you! How can I assist you today?"),
]

# 새로운 질문
question = "딥러닝 4대천황이라고 불리는 사람들은 누구야?"

chain = prompt | llm

chain.invoke({"history": history, "question": question})

AIMessage(content='딥러닝 4대 천황이라고 불리는 인물들은 보통 다음과 같은 네 명의 연구자를 가리킵니다:\n\n1. **제프리 힌턴 (Geoffrey Hinton)** - 딥러닝의 아버지로 알려져 있으며, 신경망에 대한 기초 연구를 많이 했습니다. 특히 역전파 알고리즘에 대한 연구로 유명합니다.\n\n2. **얀 르쿤 (Yann LeCun)** - CNN(합성곱 신경망)의 개척자로 잘 알려져 있으며, 이미지 인식 및 컴퓨터 비전 분야에서 중요한 기여를 했습니다.\n\n3. **요슈아 벤지오 (Yoshua Bengio)** - 딥러닝의 이론과 알고리즘 발전에 크게 기여한 연구자입니다. 특히, 심층 신경망의 학습 방법에 대한 연구로 유명합니다.\n\n4. **앤드류 응 (Andrew Ng)** - 딥러닝과 머신러닝 분야의 교육과 대중화를 이끈 인물 중 한 명입니다. Google Brain의 공동 창립자로도 알려져 있으며, Coursera의 공동 창립자이기도 합니다.\n\n이 네 명의 연구자들은 딥러닝의 발전에 중대한 기여를 했으며, 현재의 인공지능 기술 발전에 큰 영향을 미쳤습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 293, 'prompt_tokens': 62, 'total_tokens': 355, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0ba0d124f1', 'finish_